##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [25]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

###Préparation des matrices de traits

In [26]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [27]:
paradigmFile="MGC-150805-paradigmes.csv"
analysisPrefix="MGC-150805-total"
partialPrefix="MGC-150812-30000-1508120226-S"

###Préparation des cases du paradigme

In [28]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

#Préparation du calcul des analogies

###Calcul de la différence entre deux formes

In [29]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

###Accumulation des paires appartenant à un patron

In [30]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

###Transformation d'un patron en RegExp

In [31]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

###Substitution de sortie 
???

In [32]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

In [33]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print minLongueur, maxLongueur
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print phonemes
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print forme1,forme2,patron
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print patron12,patron21
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print "patron1", patron
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print "patron2", patron
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

#Classe pour la gestion des patrons, des classes et des transformations

In [34]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                print forme,
                print "pas de classe",idClasseForme,
                print "%.2f par forme de sortie" % (float(1)/len(classeForme))
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print forme, 
            print "pas de patron"
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [35]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [36]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [37]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [38]:
paradigmes=pd.read_csv(paradigmFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]

In [39]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)
with open(partialPrefix+'-Regles.pkl', 'rb') as input:
    resultatsPartiels = pickle.load(input)

In [40]:
def generateForms(lexeme,paradigmes):
    for forme in listeCases:
        if not pd.isnull(paradigmes[paradigmes["lexeme"]==lexeme][forme].iloc[0]):
            depart=paradigmes[paradigmes["lexeme"]==lexeme][forme].iloc[0]
            print forme,depart
            if depart!="nan":
                for case in listeCases:
                    print case,
                    if not isinstance(resultatsLecture[(forme, case)],str):
                        print resultatsLecture[(forme, case)].sortirForme(depart),
                print
                print

In [41]:
#generateForms(u"lever",paradigmes)

In [42]:
def showPatterns(listePaires):
    for resultat in listePaires:
        print "paire :",resultat
        for element in resultatsPartiels[resultat].patrons:
            print "\t"+element, resultatsPartiels[resultat].patrons[element]
        print

In [43]:
listePaires=[("inf","ppMS")]
showPatterns(listePaires)

paire : ('inf', 'ppMS')
	dr-zy ^(.*)kudr$
	rir-Er ^(.*[uOo][fv])rir$
	tr- ^(.*)nEtr$
	u.ir-O. ^murir$
	ivr-Eky ^(.*)vivr$
	- ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJjlrwHiyEe926uOoêûô])E$
	r-y ^(.*[ptbdfsvzrE96O][jrwHiyEe926auOoêûâô][ptkbdg])r$
	avwar-y ^(.*)avwar$
	6vwar-y ^(.*[tdsz])6vwar$
	Etr-y ^(.*[pk][E96aO][mnr])Etr$
	udr-Oly ^rEzudr$
	Er-y ^tEr$
	dr- ^(.*[ptkbdgfsSvzZjlrwH])êdr$
	watr-y ^akrwatr$
	r- ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][iyEe92])r$
	war-y ^(.*[bdvzlr])war$
	r-i ^(.*)sHivr$
	ir-y ^(.*[ptbdfsvzmnlr])ir$
	uvwar-y ^(.*)muvwar$
	i.-j. ^([pk][lr])iE$



In [44]:
paradigmes.stack().value_counts(dropna=True).sum()

333819

In [45]:
#paradigmes[paradigmes["lexeme"]==u"cuire"].stack().value_counts(dropna=True).sum()
#paradigmes[paradigmes["lexeme"]==u"dépendre"].stack()

In [46]:
"""
A dictionary difference calculator
Originally posted as:
http://stackoverflow.com/questions/1165352/fast-comparison-between-two-python-dictionary/1165552#1165552
"""


class DictDiffer(object):
    # TODO: on changed() avoid returning duplicate keys.
    # TODO:
    """
        Compares two dictionaries, traversing nested dictionaries,
        and provides four simple methods to access the differences between them:
            changed(), removed(), added(), unchanged()
         All four methods return a tuple of tuples of keys.
         When a given key is a nested one, it returns a tuple containing
         all the keys in the path to that key. For instance, given the sample above,
         the second "b" dict bellow contains the new "y" key under "x" and this in turn under "c",
         so it returns on tuple: ('c', ('x', 'y'))
    """

    def __init__(self, current_dict, past_dict):
        self._added,self._removed,self._changed,self._unchanged = self.diff_dicts(current_dict, past_dict)

        for key in (self._changed + self._unchanged):
#            print key, current_dict[key], past_dict[key]
            if isinstance(current_dict[key], dict) and isinstance(past_dict[key], dict):
                self._extend( key, self.__class__(current_dict[key], past_dict[key]) )

    def diff_dicts(self, current_dict, past_dict):
        """
        This is the original functionality for simple (not nested) dicts:
            Compares current_dict and past_dict and returns
            (added, removed, changed, unchanged) keys
        """
        set_current, set_past = set(current_dict.keys()), set(past_dict.keys())
        intersect = set_current.intersection(set_past)
        added = list(set_current - intersect)
        removed = list(set_past - intersect)
        changed = list(o for o in intersect if past_dict[o] != current_dict[o])
        unchanged = list(o for o in intersect if past_dict[o] == current_dict[o])
        return added, removed, changed, unchanged

    def _extend(self, parent, diff):
        """
            Adds the given differences to the appropriate members.
            Used with nested dicts.
        """
        self._added += [tuple([parent, o]) for o in diff._added]
        self._removed += [tuple([parent, o]) for o in diff._removed]
        self._changed += [tuple([parent, o]) for o in diff._changed]
        self._unchanged += [tuple([parent, o]) for o in diff._unchanged]

    def added(self):
        return tuple(self._added)
    def removed(self):
        return tuple(self._removed)
    def changed(self):
        return tuple(self._changed)
    def unchanged(self):
        return tuple(self._unchanged)


In [47]:
def showDifference(listePaires):
    for resultat in listePaires:
        print "paire :",resultat
    #    print resultatsLecture[resultat].patrons
    #    for element in resultatsLecture[resultat].patrons:
    #        print "\t"+element, resultatsLecture[resultat].patrons[element]
    #    print
        test=DictDiffer(resultatsPartiels[resultat].patrons,resultatsLecture[resultat].patrons)
        if test.added(): print "added",test.added()
        if test.removed():
            print "missing"
            for element in test.removed():
                print "\t"+element,resultatsLecture[resultat].patrons[element]
            print "classe"
            print "\t"+element,resultatsLecture[resultat].classe
        if test.changed():
            print "overspecified"
            for element in test.changed():
                print "\t"+element,resultatsLecture[resultat].patrons[element]
                print "\t"+element,resultatsPartiels[resultat].patrons[element]
                print
        if test.unchanged():
            print "optimal"
            print "\t"+element,resultatsLecture[resultat].patrons[element]

In [48]:
listePaires=[("inf","ppMS"),("pi3S","pi2S"),("pi3S","pi1P"),("pi3S","inf")]
showDifference(listePaires)

paire : ('inf', 'ppMS')
added (u'tr-',)
missing
	Etr-i ^(.*)mEtr$
	dr-ly ^(.*)mudr$
	war-i ^(.*[rE6a])swar$
	r-e ^Etr$
	Etr-e ^(.*)nEtr$
	re- ^kurbatyre$
	Erir-i ^(.*[E96aOêûâô])kErir$
	ksir-si ^Oksir$
	e-â ^tOne$
	Or-o ^(.*)klOr$
	e-y ^fiSe$
	âdr-i ^(.*)prâdr$
classe
	âdr-i {u'Etr-i': {u'Etr-i': 10}, u'Etr-e': {u'Etr-e': 1}, u'ir-y, r-, Erir-i': {u'Erir-i': 5}, u'ir-y, r-, rir-Er': {u'r-': 1, u'rir-Er': 10}, u'dr-, r-y, udr-Oly': {u'udr-Oly': 1}, u'dr-, dr-zy': {u'dr-zy': 3}, u'r-y, \xe2dr-i': {u'\xe2dr-i': 12}, u'dr-ly': {u'dr-ly': 3}, u'war-i, war-y': {u'war-i': 3}, u'-, re-': {'-': 1, u're-': 1}, '-': {'-': 5766}, u'6vwar-y, war-y': {u'6vwar-y': 11}, u'dr-': {u'dr-': 9}, u'Etr-i, Etr-y': {u'Etr-i': 6}, u'Or-o': {u'Or-o': 5}, u'ir-y, r-, u.ir-O.': {u'u.ir-O.': 1}, u'ir-y, r-': {u'r-': 323, u'ir-y': 43}, u'-, e-y': {u'e-y': 1, '-': 1}, u'r-y': {u'r-y': 56}, u'ir-y, ksir-si, r-': {u'ksir-si': 1}, u'watr-y': {u'watr-y': 4}, u'r-e': {u'r-e': 1}, u'-, e-\xe2': {u'e-\xe2': 1}, u'r-i': {u'